In [2]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 6.1MB/s 


In [3]:
(trainx, trainy), (testx, testy) = keras.datasets.fashion_mnist.load_data()
trainx = trainx / 255.0
testx = testx/255.0

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
def build(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  #continous values controlled by steps
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))
  # continous or discrete parameters
  # discrete-> choice
  hp_lr = hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=hp_lr),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model


In [8]:
tuner = kt.Hyperband(build,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hpdir',
                     project_name='mnisttuning')

In [9]:
# Early Stopping
# relieve algos that are wasting resources
# patience=> how many iters to ignore before checking for performance for early stopping
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [11]:
tuner.search(trainx, trainy, epochs=50, validation_split=0.2, callbacks=[earlystop])

Trial 31 Complete [00h 00m 21s]
val_accuracy: 0.8757500052452087

Best val_accuracy So Far: 0.8852499723434448
Total elapsed time: 00h 10m 23s
INFO:tensorflow:Oracle triggered exit


In [13]:
besthp = tuner.get_best_hyperparameters(num_trials=1)[0]
besthp.__dict__
# observe values-> lr-> 0.001, units=160 was best option

{'_conditions': [],
 '_hps': defaultdict(list,
             {'lr': [Choice(name: "lr", values: [0.01, 0.001, 0.0001], ordered: True, default: 0.01)],
              'units': [Int(name: "units", min_value: 32, max_value: 512, step: 32, sampling: None, default: 32)]}),
 '_name_scopes': [],
 '_space': [Int(name: "units", min_value: 32, max_value: 512, step: 32, sampling: None, default: 32),
  Choice(name: "lr", values: [0.01, 0.001, 0.0001], ordered: True, default: 0.01)],
 'values': {'lr': 0.001,
  'tuner/bracket': 2,
  'tuner/epochs': 10,
  'tuner/initial_epoch': 4,
  'tuner/round': 2,
  'tuner/trial_id': '5f9c457ac6d6ed4f965170175cfd6f75',
  'units': 160}}

In [14]:
# building a model only with the HP options!!!

model = tuner.hypermodel.build(besthp)
history = model.fit(trainx, trainy, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6500 - accuracy: 0.7691 - val_loss: 0.4681 - val_accuracy: 0.8292
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3838 - accuracy: 0.8617 - val_loss: 0.3571 - val_accuracy: 0.8735
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3524 - accuracy: 0.8742 - val_loss: 0.3745 - val_accuracy: 0.8678
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3276 - accuracy: 0.8832 - val_loss: 0.3858 - val_accuracy: 0.8699
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3157 - accuracy: 0.8862 - val_loss: 0.3531 - val_accuracy: 0.8820
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3064 - accuracy: 0.8909 - val_loss: 0.3858 - val_accuracy: 0.8777
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2954 - accuracy: 0.8962 - val_loss: 0.3605 - val_accuracy:

In [15]:
valperepoch = history.history['val_accuracy']
bestepoch = valperepoch.index(max(valperepoch)) +1 #because epochs start with 1, index starts at 0
bestepoch

33

In [17]:
# retrain the model ONLY till required epochs 
%%time
hypermodel = tuner.hypermodel.build(besthp)
history = hypermodel.fit(trainx, trainy, epochs=bestepoch, validation_split=0.2, verbose=0)
evalresults = hypermodel.evaluate(testx, testy)


313/313 [==============================] - 1s 2ms/step - loss: 0.6880 - accuracy: 0.8756
CPU times: user 2min 41s, sys: 15.4 s, total: 2min 56s
Wall time: 1min 52s


In [18]:
evalresults

[0.6880115866661072, 0.8755999803543091]